In [23]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 300)
import vcf

In [24]:
pheno = pd.read_csv('../16p12.2_rnaseq_analysis/data/pheno_final.tsv', sep='\t')
pheno = pheno.drop_duplicates('subject')
pheno = pheno.set_index('subject', drop=False)

In [47]:
# drop duplicate eQTL

perm = perm[(~perm.duplicated(['chrom', 'start']))].copy()

In [48]:
# mark minor alleles as alt
def minor_allele_is_alt(f):
    if f < 0.5:
        return True
    return False
perm['minor_allele_is_alt'] = perm.gnomad.apply(minor_allele_is_alt)

In [49]:
subs = pheno.subject.to_list()

summ = []
for sub in subs:
    for i, row in perm.iterrows():
        ens = row['ensembl']
        gt = row[sub]
        ma_is_alt = row['minor_allele_is_alt']
        
        if ma_is_alt and '1' in gt:
            summ.append([sub, ens, 'minor allele'])
        if (not ma_is_alt) and '0' in gt:
            summ.append([sub, ens, 'minor allele'])
        
summ = pd.DataFrame(summ)

In [54]:
final_sum = pd.DataFrame(summ[0].value_counts())
final_sum.columns = ['minor allele eQTL count']
final_sum['subject'] = final_sum.index.to_series()
final_sum = final_sum[['subject', 'minor allele eQTL count']].copy()

In [56]:
final_sum.to_csv('eqtl_counts.tsv', sep='\t', index=False)

In [57]:
affected_child = ['SG001', 'SG011', 'SG021', 'SG026', 'SG155', 'SG041', 'SG042', 'SG037', 'SG148', 'SG022']
carrier_parents = ['SG006', 'SG025', 'SG069', 'SG040', 'SG044', 'SG151']
noncarrier_parents = ['SG007', 'SG024', 'SG027', 'SG039', 'SG043','SG152']
grandparent = ['SG002', 'SG003', 'SG030', 'SG031', 'SG045', 'SG046']
noncarrier_sib = ['SG023', 'SG038', 'SG149', 'SG150', 'SG038']

In [60]:
final_sum[final_sum['subject'].isin(affected_child)].mean()

minor allele eQTL count    4.3
dtype: float64

In [61]:
final_sum[final_sum['subject'].isin(carrier_parents)].mean()

minor allele eQTL count    3.166667
dtype: float64